# Pruning in Pauli Propagation

This tutorial introduces the `opt` flag in `pprop` and shows how it speeds up Heisenberg-picture Pauli propagation without affecting the result.

We use the same problem as in the VQE tutorial: the 2D transverse-field Ising model on a $8 \times 8$ lattice with the same ansatz and truncation cutoffs `k1 = 6`, `k2 = 20`.

## What is pruning?

During Heisenberg evolution the `PauliDict` can grow large as each gate splits Pauli words into multiple branches. Many of these branches are guaranteed to contribute zero to the final expectation value

$$\langle 0 | O | 0 \rangle$$

because they can never reach the $Z/I$ subspace required by `to_expectation`. Pruning detects and discards these branches early, before they are evolved further and generate even more dead terms.

Note that the words carried in `PauliDict` during evolution are not the exact Heisenberg-evolved observable. They are already a truncated representation pruned by the `k1` and `k2` cutoffs. The pruning enabled by `opt=True` applies an additional structural filter on top of that, based on two criteria:

- **Dead qubit pruning**: removes any Pauli word that carries $X$ or $Y$ on a qubit that will never be touched by any remaining gate. Such a qubit is permanently frozen in a non-$ZI$ state.
- **XY-weight pruning**: removes any Pauli word whose XY-weight (number of qubits carrying $X$ or $Y$) exceeds the maximum reduction achievable by all remaining gates in the causal cone of that word.

<img src="./assets/pruning.svg" width="600">

## Step 0: Imports

In [1]:
import time

import numpy as np
import pennylane as qml

from pprop import Propagator

/home/samonaco/Pauli-Propagator/.venv/lib/python3.12/site-packages/pennylane/operation.py:2622: PennyLaneDeprecationWarning: Observable is deprecated and will be removed in v0.43. A generic Operator class should be used instead. If defining an Operator, set the is_hermitian property to True. If checking if an Operator is Hermitian, check the is_hermitian property. 
  warnings.warn(


## Step 1: Problem setup

We reuse the Hamiltonian and ansatz from the VQE tutorial. The Hamiltonian is

$$H(J, h) = -\frac{1}{N}\left(J\sum_{\langle i,j \rangle} Z_i Z_j + h\sum_i X_i\right)$$

on a $8 \times 8$ lattice with open boundary conditions, $J = h = 1$, and row-major qubit indexing.

In [2]:
side: int = 8

J: float = 1.0
h: float = 1.0
num_qubits: int = side * side


def hamiltonian(side: int, J: float, h: float) -> qml.Hamiltonian:
    coeffs = []
    obs = []
    N = side * side
    for x in range(side):
        for y in range(side):
            i = x * side + y
            if y < side - 1:
                j = x * side + (y + 1)
                coeffs.append(-J / N)
                obs.append(qml.PauliZ(i) @ qml.PauliZ(j))
            if x < side - 1:
                j = (x + 1) * side + y
                coeffs.append(-J / N)
                obs.append(qml.PauliZ(i) @ qml.PauliZ(j))
    for i in range(N):
        coeffs.append(-h / N)
        obs.append(qml.PauliX(i))
    return qml.Hamiltonian(coeffs, obs)


def circuit(params):
    index = 0
    for q in range(num_qubits):
        qml.RY(params[index], wires=q); index += 1
        qml.RX(params[index], wires=q); index += 1
    for d in range(2):
        y_start = 0 if d % 2 == 0 else 1
        for x in range(side):
            for y in range(y_start, side - 1, 2):
                qml.CNOT(wires=[x * side + y, x * side + (y + 1)])
    for q in range(num_qubits):
        qml.RX(params[index], wires=q); index += 1
    for d in range(2):
        x_start = 0 if d % 2 == 0 else 1
        for y in range(side):
            for x in range(x_start, side - 1, 2):
                qml.CNOT(wires=[x * side + y, (x + 1) * side + y])
    for q in range(num_qubits):
        qml.RX(params[index], wires=q); index += 1
    for q in range(num_qubits):
        qml.RY(params[index], wires=q); index += 1
    return qml.expval(hamiltonian(side, J, h))

## Step 2: Propagation with and without pruning

Passing `opt=True` to `.propagate()` enables both pruning strategies. We time both runs to measure the speedup.

In [3]:
prop_no_opt = Propagator(circuit, k1=6, k2=20)

t0 = time.perf_counter()
prop_no_opt.propagate(opt=False)
t_no_opt = time.perf_counter() - t0

print(f"Time without pruning : {t_no_opt:.3f} s")

prop_opt = Propagator(circuit, k1=6, k2=20)

t0 = time.perf_counter()
prop_opt.propagate(opt=True)
t_opt = time.perf_counter() - t0

print(f"Time with pruning    : {t_opt:.3f} s")
print(f"Speedup              : {t_no_opt / t_opt:.2f}x")

Propagating PauliDict(176 terms)
Time without pruning : 68.511 s
Propagating PauliDict(176 terms)
Time with pruning    : 3.866 s
Speedup              : 17.72x


## Step 3: Correctness check

Pruning must never change the result. We evaluate both propagators at the same random parameter point and assert that the outputs agree.

In [4]:
rng = np.random.default_rng(42)
params = rng.random(prop_opt.num_params)

e_no_opt = prop_no_opt(params)
e_opt    = prop_opt(params)

print(f"Without pruning : {e_no_opt}")
print(f"With pruning    : {e_opt}")

assert np.isclose(e_no_opt, e_opt, atol=1e-10), "Pruning changed the result"

print("\nOutputs match.")

Without pruning : [-0.27149064]
With pruning    : [-0.27149064]

Outputs match.
